In [1]:
# Importing dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import requests 
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

import pickle

In [ ]:
# headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
# base_url = 'https://www.presidency.ucsb.edu/documents/presidential-documents-archive-guidebook/annual-messages-congress-the-state-the-union'
# r = requests.get(base_url, headers=headers)
# driver = webdriver.Chrome()

In [ ]:
# r

In [5]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

sotu_raw_transcripts = []
presidents = []
dates = []

dates = list(range(1793, 2021)) # Dates 1793-2020
dates_str = [str(d) for d in dates]
dates_str

for date in dates_str:
    print(f'Getting transcript for {date}')
    try:
        # Opening URL
        base_url = 'https://www.presidency.ucsb.edu/documents/presidential-documents-archive-guidebook/annual-messages-congress-the-state-the-union'
        driver = webdriver.Chrome()
        driver.get(base_url)
        driver.implicitly_wait(10) # seconds
        element = driver.find_element_by_link_text(date)
        webdriver.ActionChains(driver).move_to_element(element).click(element).perform()
        current_url = driver.current_url # url containing transcript of SOTU addresses
        r = requests.get(current_url, headers=headers)
        print(r)
        # print(r.content)
        soup = BeautifulSoup(r.content, 'html.parser') 
        #print(soup.prettify())
        # Using BeautifulSoup to extract transcript
        text = soup.find('div', class_='field-docs-content').get_text()
        #text = soup.find('div', class_='node-documents').get_text()
        pres_name = soup.find('h3', class_='diet-title').get_text()
        date = soup.find('span', class_='date-display-single').get_text()
        sotu_raw_transcripts.append(text)
        presidents.append(pres_name)
        dates.append(date)
        driver.close()
    except:
        print(f'Could not get record for {date}')
#print(sotu_transcripts)
num_records = len(sotu_raw_transcripts)
print(f'Successfully scrapped {num_records} records')
driver.quit()

Getting transcript for 1793
<Response [200]>
Getting transcript for 1794
<Response [200]>
Getting transcript for 1795
<Response [200]>
Getting transcript for 1796
<Response [200]>
Getting transcript for 1797
<Response [200]>
Getting transcript for 1798
<Response [200]>
Getting transcript for 1799
<Response [200]>
Getting transcript for 1800
<Response [200]>
Getting transcript for 1801
<Response [200]>
Getting transcript for 1802
<Response [200]>
Getting transcript for 1803
<Response [200]>
Getting transcript for 1804
<Response [200]>
Getting transcript for 1805
<Response [200]>
Getting transcript for 1806
<Response [200]>
Getting transcript for 1807
<Response [200]>
Getting transcript for 1808
<Response [200]>
Getting transcript for 1809
<Response [200]>
Getting transcript for 1810
<Response [200]>
Getting transcript for 1811
<Response [200]>
Getting transcript for 1812
<Response [200]>
Getting transcript for 1813
<Response [200]>
Getting transcript for 1814
<Response [200]>
Getting tr

<Response [200]>
Getting transcript for 1975
<Response [200]>
Getting transcript for 1976
<Response [200]>
Getting transcript for 1977
<Response [200]>
Getting transcript for 1978
<Response [200]>
Getting transcript for 1979
<Response [200]>
Getting transcript for 1980
<Response [200]>
Getting transcript for 1981
<Response [200]>
Getting transcript for 1982
<Response [200]>
Getting transcript for 1983
<Response [200]>
Getting transcript for 1984
<Response [200]>
Getting transcript for 1985
<Response [200]>
Getting transcript for 1986
<Response [200]>
Getting transcript for 1987
<Response [200]>
Getting transcript for 1988
<Response [200]>
Getting transcript for 1989
<Response [200]>
Getting transcript for 1990
<Response [200]>
Getting transcript for 1991
<Response [200]>
Getting transcript for 1992
<Response [200]>
Getting transcript for 1993
<Response [200]>
Getting transcript for 1994
<Response [200]>
Getting transcript for 1995
<Response [200]>
Getting transcript for 1996
<Response 

In [6]:
# Pickling raw transcripts
with open('./pickled_files/sotu_raw_transcripts.pkl', 'wb') as f:
    pickle.dump(sotu_raw_transcripts, f)

In [7]:
# Pickling list of presidents
with open('./pickled_files/presidents.pkl', 'wb') as f:
    pickle.dump(presidents, f)

In [8]:
# Pickling list of dates
with open('./pickled_files/dates.pkl', 'wb') as f:
    pickle.dump(dates, f)

In [ ]:
# Opening picled list of raw transcripts
with open('./pickled_files/sotu_raw_transcripts.pkl', 'rb') as f:
    sotu_raw_transcripts = pickle.load(f)

In [3]:
# dates

In [4]:
# date-display-single

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

base_url = 'https://www.presidency.ucsb.edu/documents/presidential-documents-archive-guidebook/annual-messages-congress-the-state-the-union'
driver = webdriver.Chrome()
driver.get(base_url)
driver.implicitly_wait(10) # seconds
element = driver.find_element_by_link_text('1793')
webdriver.ActionChains(driver).move_to_element(element).click(element).perform()
current_url = driver.current_url # url containing transcript of SOTU addresses
r = requests.get(current_url, headers=headers)
print(r)
# print(r.content)
soup = BeautifulSoup(r.content, 'html.parser') 
#print(soup.prettify())
# Using BeautifulSoup to extract transcript
text = soup.find('div', class_='field-docs-content').get_text()
#text = soup.find('div', class_='node-documents').get_text()
pres_name = soup.find('h3', class_='diet-title').get_text()
date = soup.find('span', class_='date-display-single').get_text()

In [ ]:
pres_name

In [ ]:
date

In [ ]:
text